In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from numpy import asarray
from numpy import save
from numpy import load
import tensorflow as tf
import tensorflow.keras as keras
import pickle
import joblib
from sklearn.metrics import mean_squared_error
import seaborn as sns
from tensorflow.keras import regularizers
from sklearn.metrics import mean_squared_error
import kerastuner as kt
import scipy

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [ ]:
np.random.seed(43)
tf.random.set_seed(43)

In [ ]:
sns.set_style("white")

### Load Data

In [ ]:
with open ('../final_data/label_keys_ts.pkl', 'rb') as fp:
    label_keys = pickle.load(fp)
with open ('../final_data/feature_keys_ts.pkl', 'rb') as fp:
    feature_keys = pickle.load(fp)    

train_x = load('../final_data/train_x_norm.npy')
train_y = load('../final_data/train_y_norm.npy')
valid_x = load('../final_data/valid_x_norm.npy')
valid_y = load('../final_data/valid_y_norm.npy')
test_x = load('../final_data/test_x_norm.npy')
test_y = load('../final_data/test_y_norm.npy')
all_train_x = load('../final_data/all_train_x_norm.npy')
all_train_y = load('../final_data/all_train_y_norm.npy')

scaler_x = joblib.load("../final_data/scaler_x.save") 
scaler_y = joblib.load("../final_data/scaler_y.save") 

scaler_train_x = joblib.load("../final_data/scaler_train_x.save") 
scaler_train_y = joblib.load("../final_data/scaler_train_y.save")

train_norm = pd.read_pickle("../final_data/train_norm.pkl")
valid_norm = pd.read_pickle("../final_data/valid_norm.pkl")
test_norm = pd.read_pickle("../final_data/test_norm.pkl")
all_train_norm = pd.read_pickle("../final_data/all_train_norm.pkl")

## Loop function

In [ ]:
def test_5_times(model, epochs, batch_size):
    runs = 5
    test_score = list()
    train_score = list()
 
    for i in range(runs):

        np.random.seed(42 + i)
        tf.random.set_seed(42 + i)



        history = model.fit(all_train_x, all_train_y,
                        epochs=epochs,
                        batch_size=batch_size,
                       )

        y_pred_test = model.predict(test_x)
        y_test_unsc = scaler_y.inverse_transform(test_y)
        y_pred_test_unsc = scaler_y.inverse_transform(y_pred_test)

        y_pred_train = model.predict(all_train_x)
        y_train_unsc = scaler_y.inverse_transform(all_train_y)
        y_pred_train_unsc = scaler_y.inverse_transform(y_pred_train)
        
        mse_test = np.sqrt(mean_squared_error(y_pred_test_unsc, y_test_unsc))  
        test_score.append(mse_test)

        mse_train = np.sqrt(mean_squared_error(y_pred_train_unsc, y_train_unsc))  
        train_score.append(mse_train)

    return  train_score, test_score


## Simple LSTM

In [ ]:
def build_simple_lstm():
    LSTMSimple_model = keras.models.Sequential([
    keras.layers.LSTM(32,
                      input_shape=(all_train_x.shape[1],all_train_x.shape[2]),
                      return_sequences=False),
    
    keras.layers.Dense(2, kernel_initializer='he_normal', activation='relu')
    ])

    LSTMSimple_model.compile(loss='mean_squared_error', optimizer=keras.optimizers.Adam(lr=1e-3))
    LSTMSimple_model.summary()
    
    return LSTMSimple_model

### Training

In [ ]:
LSTMSimple_model = build_simple_lstm()
LSTMSimple_model_history = LSTMSimple_model.fit(train_x, train_y,
                    epochs=200,
                    batch_size=128, 
                    validation_data=(valid_x, valid_y),
                    verbose=2
                   )       

Load model

In [ ]:
LSTMSimple_model_train = keras.models.load_model("../final_data/models/LSTMSimple_model")
hist_simple_lstm = pd.read_pickle("../final_data/hist_simple_lstm.pkl")

In [ ]:
## Calculate RMSE on training set
y_pred_train = LSTMSimple_model_train.predict(train_x)
y_train_unsc = scaler_train_y.inverse_transform(train_y)
y_pred_train_unsc = scaler_train_y.inverse_transform(y_pred_train)
np.sqrt(mean_squared_error(y_train_unsc, y_pred_train_unsc))                       

In [ ]:
y_pred_valid= LSTMSimple_model_train.predict(valid_x)
y_valid_unsc = scaler_train_y.inverse_transform(valid_y)
y_pred_valid_unsc = scaler_train_y.inverse_transform(y_pred_valid)

np.sqrt(mean_squared_error(y_pred_valid_unsc, y_valid_unsc))  

In [ ]:
pd.DataFrame(hist_simple_lstm).plot(figsize=(8, 5))
plt.grid(True)
plt.show()

## Testing

In [ ]:
LSTMSimple_model_test = build_simple_lstm()
LSTMSimple_model_history = LSTMSimple_model_test.fit(all_train_x, all_train_y,
                    epochs=75,
                    batch_size=128, 
                    verbose=2
                   )             

Load model

In [ ]:
LSTMSimple_model_test = keras.models.load_model("../final_data/models/LSTMSimple_model_test")

In [ ]:
## Calculate RMSE on all training data
y_pred_all_train = LSTMSimple_model_test.predict(all_train_x)
y_all_train_unsc = scaler_y.inverse_transform(all_train_y)
y_pred_all_train_unsc = scaler_y.inverse_transform(y_pred_all_train)
np.sqrt(mean_squared_error(y_all_train_unsc, y_pred_all_train_unsc))         

In [ ]:
y_pred_test= LSTMSimple_model_test.predict(test_x)
y_test_unsc = scaler_y.inverse_transform(test_y)
y_pred_test_unsc = scaler_y.inverse_transform(y_pred_test)
np.sqrt(mean_squared_error(y_pred_test_unsc, y_test_unsc))  

## Test 5 times

In [ ]:
LSTMSimple_model = build_simple_lstm()
epochs = 75
batch_size = 128
train_score, test_score = test_5_times(LSTMSimple_model, epochs, batch_size)

In [ ]:
rmse_train_simple_lstm = pd.read_pickle("../final_data/rmse_train_simple_lstm.pkl")
rmse_test_simple_lstm = pd.read_pickle("../final_data/rmse_test_simple_lstm.pkl")

In [ ]:
rmse_train_simple_lstm.describe()

In [ ]:
rmse_test_simple_lstm.describe()

## Simple LSTM with Dropout

In [ ]:
Simple_Dropout_model = keras.models.Sequential([
        keras.layers.LSTM(32,
                          input_shape=(all_train_x.shape[1],all_train_x.shape[2]),
                          return_sequences=False,
                          dropout=0.2,
                          recurrent_dropout=0.2,
                          ),
        keras.layers.Dense(2, kernel_initializer='he_normal', activation='relu'),
    ])

Simple_Dropout_model.compile(loss='mean_squared_error', optimizer=keras.optimizers.Adam(lr=1e-3))
Simple_Dropout_model.summary()
    
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=50)

Simple_Dropout_model_history = Simple_Dropout_model.fit(train_x, train_y,
                    epochs=200,
                    batch_size=128, 
                    validation_data=(valid_x, valid_y),
                    callbacks=[callback,],
                    verbose=2
                   )

Load model

In [ ]:
hist_simple_dropout = pd.read_pickle("../final_data/hist_simple_lstm_dropout.pkl")
LSTMSimple_dropout = keras.models.load_model("../final_data/models/LSTMSimple_dropout")

In [ ]:
LSTMSimple_dropout.summary()

In [ ]:
pd.DataFrame(hist_simple_dropout).plot(figsize=(8, 5))
plt.grid(True)
plt.show()

In [ ]:
## Calculate RMSE on training set
y_pred_train = LSTMSimple_dropout.predict(train_x)
y_train_unsc = scaler_train_y.inverse_transform(train_y)
y_pred_train_unsc = scaler_train_y.inverse_transform(y_pred_train)
np.sqrt(mean_squared_error(y_train_unsc, y_pred_train_unsc))                       

In [ ]:
y_pred_valid= LSTMSimple_dropout.predict(valid_x)
y_valid_unsc = scaler_train_y.inverse_transform(valid_y)
y_pred_valid_unsc = scaler_train_y.inverse_transform(y_pred_valid)

np.sqrt(mean_squared_error(y_pred_valid_unsc, y_valid_unsc))  

## Simple LSTM with Weight Regularization

In [ ]:
Simple_Weight_model = keras.models.Sequential([
        keras.layers.LSTM(32,
                          input_shape=(all_train_x.shape[1],all_train_x.shape[2]),
                          return_sequences=False,
                          dropout=0.2,
                          recurrent_dropout=0.2,
                          ),
        keras.layers.Dense(2, kernel_initializer='he_normal', activation='relu'),
    ])

Simple_Weight_model.compile(loss='mean_squared_error', optimizer=keras.optimizers.Adam(lr=1e-3))
Simple_Weight_model.summary()
    
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=50)

Simple_Weight_model_history = Simple_Weight_model.fit(train_x, train_y,
                    epochs=500,
                    batch_size=128, 
                    validation_data=(valid_x, valid_y),
                    callbacks=[callback,],
                    verbose=2
                   )

Load model

In [ ]:
hist_simple_weight_reg = pd.read_pickle("../final_data/hist_simple_lstm_weight_reg.pkl")
LSTMSimple_weight_reg = keras.models.load_model("../final_data/models/LSTMSimple_weight_reg")

In [ ]:
pd.DataFrame(hist_simple_weight_reg).plot(figsize=(8, 5))
plt.grid(True)
plt.show()

In [ ]:
## Calculate RMSE on training set
y_pred_train = LSTMSimple_weight_reg.predict(train_x)
y_train_unsc = scaler_train_y.inverse_transform(train_y)
y_pred_train_unsc = scaler_train_y.inverse_transform(y_pred_train)
np.sqrt(mean_squared_error(y_train_unsc, y_pred_train_unsc))                       

In [ ]:
y_pred_valid= LSTMSimple_weight_reg.predict(valid_x)
y_valid_unsc = scaler_train_y.inverse_transform(valid_y)
y_pred_valid_unsc = scaler_train_y.inverse_transform(y_pred_valid)

np.sqrt(mean_squared_error(y_pred_valid_unsc, y_valid_unsc))  

## LSTM with Dropout

### Define Model

In [ ]:
def build_lstm_dropout():
    LSTMDropout_model = keras.models.Sequential([
        keras.layers.LSTM(16,
                          input_shape=(all_train_x.shape[1],all_train_x.shape[2]),
                          return_sequences=True,
                          dropout=0.3,
                          recurrent_dropout=0.3,
                          kernel_constraint=keras.constraints.max_norm(max_value=1),
                          recurrent_constraint=keras.constraints.max_norm(max_value=1),
                          ),

        keras.layers.LSTM(16,
                        return_sequences=True,
                        dropout=0.3,
                        recurrent_dropout=0.3,
                        kernel_constraint=keras.constraints.max_norm(max_value=1),
                        recurrent_constraint=keras.constraints.max_norm(max_value=1),
                         ),
        
        keras.layers.LSTM(16,
                        return_sequences=False,
                        dropout=0.3,
                        recurrent_dropout=0.3,
                        kernel_constraint=keras.constraints.max_norm(max_value=1),
                        recurrent_constraint=keras.constraints.max_norm(max_value=1),
                        ),

        keras.layers.Dense(2, kernel_initializer='he_normal', activation='relu'),
    ])

    LSTMDropout_model.compile(loss='mean_squared_error', optimizer=keras.optimizers.Adam(lr=1e-3))
    LSTMDropout_model.summary()
    
    return LSTMDropout_model

### Training

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
LSTMDropout_model_train = build_lstm_dropout()
LSTMDropout_history = LSTMDropout_model_train.fit(train_x, train_y,
                    epochs=100,
                    batch_size=128, 
                    validation_data=(valid_x, valid_y),
                    callbacks=[
                              callback,
                              ],
                    verbose=2
                   )

In [ ]:
LSTMDropout_model_train = keras.models.load_model("../final_data/models/LSTMDropout_model_train")

In [ ]:
y_pred_train = LSTMDropout_model_train.predict(train_x)
y_train_unsc = scaler_train_y.inverse_transform(train_y)
y_pred_train_unsc = scaler_train_y.inverse_transform(y_pred_train)
np.sqrt(mean_squared_error(y_train_unsc, y_pred_train_unsc))                       

In [ ]:
## Predict Validation Set and calculate RMSE
y_pred_valid= LSTMDropout_model_train.predict(valid_x)
y_valid_unsc = scaler_train_y.inverse_transform(valid_y)
y_pred_valid_unsc = scaler_train_y.inverse_transform(y_pred_valid)

np.sqrt(mean_squared_error(y_pred_valid_unsc, y_valid_unsc))  

In [ ]:
hist_LSTMDropout_train = pd.read_pickle("../final_data/hist_LSTMDropout_train")

pd.DataFrame(hist_LSTMDropout_train).plot(figsize=(8, 5))
plt.grid(True)
plt.show()

### Testing

In [ ]:
LSTMDropout_model_test = build_lstm_dropout()
LSTMDropout_history = LSTMDropout_model_test.fit(all_train_x, all_train_y,
                    epochs=99,
                    batch_size=128, 
                    verbose=2
                   )

Load final model

In [ ]:
LSTMDropout_model_test = keras.models.load_model("../final_data/models/LSTMDropout_model_test")

In [ ]:
y_pred_all_train = LSTMDropout_model_test.predict(all_train_x)
y_all_train_unsc = scaler_y.inverse_transform(all_train_y)
y_pred_all_train_unsc = scaler_y.inverse_transform(y_pred_all_train)
np.sqrt(mean_squared_error(y_all_train_unsc, y_pred_all_train_unsc))         

In [ ]:
y_pred_test= LSTMDropout_model_test.predict(test_x)
y_test_unsc = scaler_y.inverse_transform(test_y)
y_pred_test_unsc = scaler_y.inverse_transform(y_pred_test)

np.sqrt(mean_squared_error(y_pred_test_unsc, y_test_unsc))  

### Test 5 times with different seeds

In [ ]:
LSTMDropout_model = build_lstm_dropout()
epochs = 99
batch_size = 128
train_score, test_score = test_5_times(LSTMDropout_model, epochs, batch_size)

In [ ]:
rmse_train_lstm_dropout = pd.read_pickle("../final_data/rmse_train_lstm_dropout.pkl")
rmse_test_lstm_dropout = pd.read_pickle("../final_data/rmse_test_lstm_dropout.pkl")

In [ ]:
rmse_train_lstm_dropout.describe()

In [ ]:
rmse_test_lstm_dropout.describe()

### LSTM with weight regularization

### Build Model

In [ ]:
def build_lstm_weight_reg():
    LSTMWeightDecay_model = keras.models.Sequential([
        keras.layers.LSTM(32,
                          input_shape=(train_x.shape[1],train_x.shape[2]),
                          return_sequences=True,
                          kernel_regularizer=regularizers.l2(0.0001),
                          recurrent_regularizer=regularizers.l2(0.0001),
                          ),


        keras.layers.LSTM(32,
                        return_sequences=False,
                        kernel_regularizer=regularizers.l2(0.0001),
                        recurrent_regularizer=regularizers.l2(0.0001),
                        ),

        keras.layers.Dense(2, kernel_initializer='he_normal')
    ])

    LSTMWeightDecay_model.compile(loss='mean_squared_error', optimizer=keras.optimizers.Adam(lr=1e-3))
    LSTMWeightDecay_model.summary()
    
    return LSTMWeightDecay_model

### Training

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
LSTMWeightDecay_model_train = build_lstm_weight_reg()
LSTMWeightDecay_history = LSTMWeightDecay_model.fit(train_x, train_y,
                    epochs=300,
                    batch_size=128, 
                    validation_data=(valid_x, valid_y),
                    callbacks=[callback],          
                    verbose=2
                   )

In [ ]:
LSTMWeightDecay_model_train = keras.models.load_model("../final_data/models/LSTMWeightDecay_model_train")
hist_LSTMWeightDecay_train = pd.read_pickle("../final_data/hist_LSTMWeightDecay_train.pkl")

### Performance

In [ ]:
pd.DataFrame(hist_LSTMWeightDecay_train).plot(figsize=(8, 5))
plt.grid(True)
plt.show()

In [ ]:
## Predict Validation Set and calculate RMSE
y_pred_valid= LSTMWeightDecay_model_train.predict(valid_x)
y_valid_unsc = scaler_train_y.inverse_transform(valid_y)
y_pred_valid_unsc = scaler_train_y.inverse_transform(y_pred_valid)

np.sqrt(mean_squared_error(y_pred_valid_unsc, y_valid_unsc))  

### Testing

In [ ]:
LSTMWeightDecay_model = build_lstm_weight_reg()
LSTMWeightDecay_history = LSTMWeightDecay_model.fit(all_train_x, all_train_y,
                    epochs=230,
                    batch_size=128,        
                    verbose=2
                   )

In [ ]:
LSTMWeightDecay_model = keras.models.load_model("../final_data/models/LSTMWeightDecay_test")

In [ ]:
y_pred_all_train = LSTMWeightDecay_model.predict(all_train_x)
y_all_train_unsc = scaler_y.inverse_transform(all_train_y)
y_pred_all_train_unsc = scaler_y.inverse_transform(y_pred_all_train)
np.sqrt(mean_squared_error(y_all_train_unsc, y_pred_all_train_unsc))         

In [ ]:
## Predicit test set and calculate RMSE
y_pred_test= LSTMWeightDecay_model.predict(test_x)
y_test_unsc = scaler_y.inverse_transform(test_y)
y_pred_test_unsc = scaler_y.inverse_transform(y_pred_test)

np.sqrt(mean_squared_error(y_pred_test_unsc, y_test_unsc))  

### Test 5 times with different seeds

In [ ]:
LSTMWeightDecay_model = build_lstm_weight_reg()
epochs = 230
batch_size = 128
train_score, test_score = test_5_times(LSTMWeightDecay_model, epochs, batch_size)

In [ ]:
rmse_train_lstm_weight = pd.read_pickle("../final_data/rmse_train_lstm_weight.pkl")
rmse_test_lstm_weight = pd.read_pickle("../final_data/rmse_test_lstm_weight.pkl")

In [ ]:
rmse_train_lstm_weight.describe()

In [ ]:
rmse_test_lstm_weight.describe()

## LSTM with Hyperband

### Keras Tuner implementation of Hyperband
Hyperparameter search was interrupted after 2 weeks of searching. There is no final tuner to load, the best model found was manually rebuilt.

In [ ]:
# ## Model architecture and Hyperparameter with search spaces
# def model_builder (hp):
    
#     model = keras.models.Sequential()
    
#     ## Define Hyperparameter search space
#     hp_units = hp.Int("unit",min_value=30,max_value=320,step=32)
#     hp_dropout=hp.Choice("dropout",values = [0.2, 0.3, 0.4])  
#     hp_number_of_layers = hp.Int("nr_layers",min_value=3,max_value=8,step=1)
    
#     model.add(keras.layers.LSTM(
#                         units=hp_units, 
#                         dropout=hp_dropout,          
#                         recurrent_dropout=hp_dropout,                       
#                         return_sequences=True,
#                         input_shape=(train_x.shape[1],train_x.shape[2]),  
#                         kernel_constraint=keras.constraints.max_norm(max_value=1),
#                         recurrent_constraint=keras.constraints.max_norm(max_value=1),
#                  ))
    
#     for i in range(hp_number_of_layers):
#         model.add(keras.layers.LSTM(
#                         units=hp_units, 
#                         dropout=hp_dropout,          
#                         recurrent_dropout=hp_dropout, 
#                         return_sequences=True,
#                         kernel_constraint=keras.constraints.max_norm(max_value=1),
#                         recurrent_constraint=keras.constraints.max_norm(max_value=1),
        
#         ))
        
        
        
#     model.add(keras.layers.LSTM(
#                         units=hp_units, 
#                         dropout=hp_dropout,          
#                         recurrent_dropout=hp_dropout, 
#                         return_sequences=False,  
#                         kernel_constraint=keras.constraints.max_norm(max_value=1),
#                         recurrent_constraint=keras.constraints.max_norm(max_value=1),
#                  ))
    
 
#     model.add(keras.layers.Dense(2, kernel_initializer='he_normal'))
        
#     hp_learning_rate = hp.Choice('learning_rate', values = [1e-3, 1e-4])
#     optimizer = keras.optimizers.Adam(lr=hp_learning_rate)
    
#     model.compile(loss='mean_squared_error', optimizer=optimizer)
#     return model



# ## Let Hyperband also search for best batch size
# class MyTuner(kt.tuners.Hyperband):
#     def run_trial(self, trial, *args, **kwargs):
#         kwargs['batch_size'] = trial.hyperparameters.Choice("batch_size",values = [32, 128, 512])
#         super(MyTuner, self).run_trial(trial, *args, **kwargs)

# import os

# ## Let Hyperband run
# tuner = MyTuner(model_builder,
#     objective='val_loss',
#     max_epochs = 50,
#     factor = 3,
#     seed = 42,
#     directory = '../data/keras_tuner', 
#     project_name = 'tuner_lstm')               
#                                        

In [ ]:
# stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
# tuner.search(train_x,
#     train_y,
#     epochs = 50,
#     validation_data = (valid_x, valid_y),
#     callbacks = [stop_early]
# )


Save tuner after search is completed

In [ ]:
# with open("../data/keras_tuner/tuner_lstm.pkl", 'wb') as f:
#     pickle.dump(tuner, f)

In [ ]:
# tuner_lstm = pickle.load(open("../data/keras_tuner/tuner_lstm.pkl",'rb'))
# print(tuner_lstm.get_best_hyperparameters()[0].values)

### Training

#### Either: Rebuild best model with tuned hyperparameters and find best number of epochs
After 2 weeks of searching, best model found was manually rebuilt, therefore there is no tuner to load.

In [ ]:
# best_hps = tuner_lstm.get_best_hyperparameters(num_trials=1)[0]
# hypermodel_lstm = tuner_lstm.hypermodel.build(best_hps)
# early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
# # Retrain the model
# hypermodel_lstm_history = hypermodel_lstm.fit(train_x, train_y,
#                     epochs=100,
#                     batch_size=32,
#                     validation_data=(valid_x, valid_y),
#                     callbacks=[early_stop]
#                    )

#### Rebuild best Hyperband model manually

In [ ]:
def build_hyperband_lstm():
    hypermodel_lstm = keras.models.Sequential([
        keras.layers.LSTM(94,
                          input_shape=(train_x.shape[1],train_x.shape[2]),
                          return_sequences=True,
                          dropout=0.2,
                          recurrent_dropout=0.2,
                          kernel_constraint=keras.constraints.max_norm(max_value=1),
                          recurrent_constraint=keras.constraints.max_norm(max_value=1)
                          ),

        keras.layers.LSTM(94,
                          dropout=0.2,
                          recurrent_dropout=0.2, 
                          return_sequences=True,
                          kernel_constraint=keras.constraints.max_norm(max_value=1),
                          recurrent_constraint=keras.constraints.max_norm(max_value=1)
                          ),

        keras.layers.LSTM(94,
                          dropout=0.2,
                          recurrent_dropout=0.2, 
                          return_sequences=True,
                          kernel_constraint=keras.constraints.max_norm(max_value=1),
                          recurrent_constraint=keras.constraints.max_norm(max_value=1)
                          ),

        keras.layers.LSTM(94,
                          dropout=0.2,
                          recurrent_dropout=0.2, 
                          return_sequences=True,
                          kernel_constraint=keras.constraints.max_norm(max_value=1),
                          recurrent_constraint=keras.constraints.max_norm(max_value=1)
                          ),


         keras.layers.LSTM(94,
                          dropout=0.2,
                          recurrent_dropout=0.2, 
                          return_sequences=False,
                          kernel_constraint=keras.constraints.max_norm(max_value=1),
                          recurrent_constraint=keras.constraints.max_norm(max_value=1)
                          ),

       keras.layers.Dense(2, kernel_initializer='he_normal')
    ])

    hypermodel_lstm.compile(loss='mean_squared_error', optimizer=keras.optimizers.Adam(lr=1e-3))
    hypermodel_lstm.summary() 
    return hypermodel_lstm


In [ ]:
LSTMHyperband_train = keras.models.load_model("../final_data/models/LSTMHyperband_train")
hist_LSTMHyperband_train = pd.read_pickle("../final_data/hist_LSTMHyperband_train.pkl")

#### Performance

In [ ]:
pd.DataFrame(hist_LSTMHyperband_train).plot(figsize=(8, 5))
plt.grid(True)
plt.show()

In [ ]:
## Predict Validation Set and calculate RMSE
y_pred_valid= LSTMHyperband_train.predict(valid_x)
y_valid_unsc = scaler_train_y.inverse_transform(valid_y)
y_pred_valid_unsc = scaler_train_y.inverse_transform(y_pred_valid)

np.sqrt(mean_squared_error(y_pred_valid_unsc, y_valid_unsc))  

### Testing
#### Either: rebuild Keras Tuner model

In [ ]:
# best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
# hypermodel_lstm = tuner.hypermodel.build(best_hps)
# # Retrain the model
# history = hypermodel_lstm.fit(all_train_x, all_train_y,
#                     epochs=31,
#                     batch_size=32,
#                    )

#### Or: rebuild best model manually

In [ ]:
LSTMHyperband_test = build_hyperband_lstm()
hypermodel_lstm_history = LSTMHyperband_test.fit(all_train_x, all_train_y,
                    epochs=31,
                    batch_size=128,
                   )

In [ ]:
LSTMHyperband_test = keras.models.load_model("../final_data/models/hypermodel_lstm")

In [ ]:
y_pred_all_train = LSTMHyperband_test.predict(all_train_x)
y_all_train_unsc = scaler_y.inverse_transform(all_train_y)
y_pred_all_train_unsc = scaler_y.inverse_transform(y_pred_all_train)
np.sqrt(mean_squared_error(y_all_train_unsc, y_pred_all_train_unsc))         

In [ ]:
## Predicit test set and calculate RMSE
y_pred_test= LSTMHyperband_test.predict(test_x)
y_test_unsc = scaler_y.inverse_transform(test_y)
y_pred_test_unsc = scaler_y.inverse_transform(y_pred_test)

np.sqrt(mean_squared_error(y_pred_test_unsc, y_test_unsc))  

### Test 5 times with different seeds

In [ ]:
hypermodel_lstm = build_hyperband_lstm()
epochs = 31
batch_size = 128
train_score, test_score = test_5_times(hypermodel_lstm, epochs, batch_size)

In [ ]:
rmse_train_lstm_hyperband = pd.read_pickle("../final_data/rmse_train_lstm_hyperband.pkl")
rmse_test_lstm_hyperband = pd.read_pickle("../final_data/rmse_test_lstm_hyperband.pkl")

In [ ]:
rmse_train_lstm_hyperband.describe()

In [ ]:
rmse_test_lstm_hyperband.describe()

### PeepholeDropout

In [ ]:
def build_peephole_dropout():
    PeepholeDropout_model = keras.models.Sequential([

        keras.layers.RNN(tf.keras.experimental.PeepholeLSTMCell(16, 
                         dropout=0.3,
                         recurrent_dropout=0.3,
                         kernel_constraint=keras.constraints.max_norm(max_value=1),
                         recurrent_constraint=keras.constraints.max_norm(max_value=1)
                                                                ),
                          return_sequences=True,
                        input_shape=(train_x.shape[1],train_x.shape[2]),
                        ),
         keras.layers.RNN(tf.keras.experimental.PeepholeLSTMCell(16, 
                         dropout=0.3,
                         recurrent_dropout=0.3,
                         kernel_constraint=keras.constraints.max_norm(max_value=1),
                         recurrent_constraint=keras.constraints.max_norm(max_value=1)
                                                                ),
                          return_sequences=True),

         keras.layers.RNN(tf.keras.experimental.PeepholeLSTMCell(16, 
                         dropout=0.3,
                         recurrent_dropout=0.3,
                         kernel_constraint=keras.constraints.max_norm(max_value=1),
                         recurrent_constraint=keras.constraints.max_norm(max_value=1),
                                                                ),
                          return_sequences=False),

        keras.layers.Dense(2, kernel_initializer='he_normal')
    ])

    PeepholeDropout_model.compile(loss='mean_squared_error', optimizer=keras.optimizers.Adam(lr=1e-3))                               
    PeepholeDropout_model.summary()
    
    return PeepholeDropout_model

### Training

In [ ]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
PeepholeDropout_model = build_peephole_dropout()
PeepholeDropout_model_history = PeepholeDropout_model.fit(train_x, train_y,
                    epochs=100,
                    batch_size=128, 
                    validation_data=(valid_x, valid_y),
                    callbacks=[callback],
                    verbose=2
                   )

In [ ]:
PeepholeDropout_model = keras.models.load_model("../final_data/models/PeepholeDropout_model_train")
hist_PeepholeDropout_train = pd.read_pickle("../final_data/hist_PeepholeDropout_model_train.pkl")

#### Performance

In [ ]:
pd.DataFrame(hist_PeepholeDropout_train).plot(figsize=(8, 5))
plt.grid(True)
plt.show()

In [ ]:
## Predict Validation Set and calculate RMSE
y_pred_valid= peephole_3.predict(valid_x)
y_valid_unsc = scaler_train_y.inverse_transform(valid_y)
y_pred_valid_unsc = scaler_train_y.inverse_transform(y_pred_valid)

np.sqrt(mean_squared_error(y_pred_valid_unsc, y_valid_unsc))  

### Testing

In [ ]:
PeepholeDropout_model = build_peephole_dropout()
PeepholeDropout_model_history = PeepholeDropout_model.fit(all_train_x, all_train_y,
                    epochs=53,
                    batch_size=128, 
                    verbose=2
                   )

In [ ]:
PeepholeDropout_model = keras.models.load_model("../final_data/models/PeepholeDropout_model_2")

In [ ]:
y_pred_all_train = PeepholeDropout_model.predict(all_train_x)
y_all_train_unsc = scaler_y.inverse_transform(all_train_y)
y_pred_all_train_unsc = scaler_y.inverse_transform(y_pred_all_train)
np.sqrt(mean_squared_error(y_all_train_unsc, y_pred_all_train_unsc))         

In [ ]:
## Predicit test set and calculate RMSE
y_pred_test= PeepholeDropout_model.predict(test_x)
y_test_unsc = scaler_y.inverse_transform(test_y)
y_pred_test_unsc = scaler_y.inverse_transform(y_pred_test)

np.sqrt(mean_squared_error(y_pred_test_unsc, y_test_unsc))  

### Test 5 times with different seeds

In [ ]:
PeepholeDropout_model = build_peephole_dropout()
epochs = 53
batch_size = 128
train_score, test_score = test_5_times(PeepholeDropout_model, epochs, batch_size)

In [ ]:
rmse_train_peephole = pd.read_pickle("../final_data/rmse_train_peephole.pkl")
rmse_test_peephole = pd.read_pickle("../final_data/rmse_test_peephole.pkl")

In [ ]:
rmse_train_peephole.describe()

In [ ]:
rmse_test_peephole.describe()

### PeepholeHyperband
We use the same architecture we found using Hyperband for the standard LSTM modell

### Training

In [ ]:
def build_hyperband_peephole():
    callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10)

    model = keras.models.Sequential([

        keras.layers.RNN(tf.keras.experimental.PeepholeLSTMCell(94, 
                         dropout=0.3,
                         recurrent_dropout=0.3,
                         kernel_constraint=keras.constraints.max_norm(max_value=4),
                         recurrent_constraint=keras.constraints.max_norm(max_value=4),
                                                                ),
                          return_sequences=True,
                        input_shape=(train_x.shape[1],train_x.shape[2]),
                        ),

         keras.layers.RNN(tf.keras.experimental.PeepholeLSTMCell(94, 
                         dropout=0.3,
                         recurrent_dropout=0.3,
                         kernel_constraint=keras.constraints.max_norm(max_value=4),
                         recurrent_constraint=keras.constraints.max_norm(max_value=4),
                                                                ),
                          return_sequences=True),


         keras.layers.RNN(tf.keras.experimental.PeepholeLSTMCell(94, 
                         dropout=0.3,
                         recurrent_dropout=0.3,
                        kernel_constraint=keras.constraints.max_norm(max_value=4),
                         recurrent_constraint=keras.constraints.max_norm(max_value=4),                                        
                                                                ),
                          return_sequences=True),

          keras.layers.RNN(tf.keras.experimental.PeepholeLSTMCell(94, 
                         dropout=0.3,
                         recurrent_dropout=0.3,
                         kernel_constraint=keras.constraints.max_norm(max_value=4),
                         recurrent_constraint=keras.constraints.max_norm(max_value=4),
                                                                ),
                          return_sequences=True),



        keras.layers.RNN(tf.keras.experimental.PeepholeLSTMCell(94, 
                         dropout=0.3,
                         recurrent_dropout=0.3,
                        kernel_constraint=keras.constraints.max_norm(max_value=4),
                         recurrent_constraint=keras.constraints.max_norm(max_value=4),                                       
                                                                ),
                          return_sequences=False),

        keras.layers.Dense(2, kernel_initializer='he_normal')
    ])

    model.compile(loss='mean_squared_error', optimizer=keras.optimizers.Adam(lr=1e-3), metrics=['mae'])
    model.summary() 
    return hypermodel_lstm


In [ ]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
PeepholeHyperband_train = build_hyperband_peephole()
hypermodel_peephole_history = PeepholeHyperband_train.fit(train_x, train_y,
                    epochs=100,
                    batch_size=128,
                    validation_data=(valid_x, valid_y),
                    callbacks=[early_stop]
                   )

In [ ]:
PeepholeHyperband_train = keras.models.load_model("../final_data/models/PeepholeHyperband_train")
hist_PeepholeHyperband = pd.read_pickle("../final_data/hist_PeepholeHyperband_train.pkl")

#### Performance

In [ ]:
pd.DataFrame(hist_PeepholeHyperband).plot(figsize=(8, 5))
plt.grid(True)
plt.show()

In [ ]:
## Predict Validation Set and calculate RMSE
y_pred_valid= PeepholeHyperband_model_test.predict(valid_x)
y_valid_unsc = scaler_train_y.inverse_transform(valid_y)
y_pred_valid_unsc = scaler_train_y.inverse_transform(y_pred_valid)

np.sqrt(mean_squared_error(y_pred_valid_unsc, y_valid_unsc))  

### Testing

In [ ]:
hypermodel_peephole = build_hyperband_lstm()
hypermodel_peephole_history = hypermodel_peephole.fit(all_train_x, all_train_y,
                    epochs=28,
                    batch_size=128,
                   )

In [ ]:
PeepholeHyperband_test = keras.models.load_model("../final_data/models/hypermodel_peephole")

In [ ]:
y_pred_all_train = PeepholeHyperband_test.predict(all_train_x)
y_all_train_unsc = scaler_y.inverse_transform(all_train_y)
y_pred_all_train_unsc = scaler_y.inverse_transform(y_pred_all_train)
np.sqrt(mean_squared_error(y_all_train_unsc, y_pred_all_train_unsc))         

In [ ]:
## Predicit test set and calculate RMSE
y_pred_test= PeepholeHyperband_test.predict(test_x)
y_test_unsc = scaler_y.inverse_transform(test_y)
y_pred_test_unsc = scaler_y.inverse_transform(y_pred_test)

np.sqrt(mean_squared_error(y_pred_test_unsc, y_test_unsc))  

### Test 5 times with different seeds

In [ ]:
hypermodel_peephole = build_hyperband_lstm()
epochs = 28
batch_size = 128
train_score, test_score = test_5_times(hypermodel_peephole, epochs, batch_size)

In [ ]:
rmse_train_peep_hyper = pd.read_pickle("/home/di40438/bachelorarbeit/final_data/rmse_train_peep_hyper.pkl")
rmse_test_peep_hyper = pd.read_pickle("/home/di40438/bachelorarbeit/final_data/rmse_test_peep_hyper.pkl")

In [ ]:
rmse_train_peep_hyper.describe()

In [ ]:
rmse_test_peep_hyper.describe()

# Graphics

In [ ]:
sns.set_style("white")

### Plot Simple LSTMs

#### If not all models were run: Read saved files

In [ ]:
hist_simple_lstm = pd.read_pickle("../final_data/hist_simple_lstm.pkl")
hist_simple_dropout = pd.read_pickle("../final_data/hist_simple_lstm_dropout.pkl")
hist_simple_weight_reg = pd.read_pickle("../final_data/hist_simple_lstm_weight_reg.pkl")

In [ ]:
fig_simple_lstms, ax = plt.subplots(figsize=(12, 5))
ax.plot(hist_simple_lstm['loss'], label='SimpleLSTM_Loss', color='dimgrey' )
ax.plot(hist_simple_lstm['val_loss'], label='SimpleLSTM_Validation_Loss', color='black' )
ax.plot(hist_simple_dropout['loss'], label='SimpleDropout_Loss', color='indianred')
ax.plot(hist_simple_dropout['val_loss'], label='SimpleDropout_Validation_Loss', color='red')
ax.plot(hist_simple_weight_reg['loss'], label='SimpleWeightReg_Loss', color='c')
ax.plot(hist_simple_weight_reg['val_loss'], label='SimpleWeightReg_Validation_Loss', color='blue')
ax.set_ylim(0.0, 0.004)
ax.set(xlabel=' Epochs', ylabel='Score')
ax.set_xlabel('Epochs', fontsize=15)
ax.set_ylabel('Score', fontsize=15)
plt.yticks(fontsize=13)
plt.xticks(fontsize=13)
ax.legend(loc='upper right',prop={'size': 10} )
ax.grid()

### Plot Tunded LSTMs

#### If not all models were run: Read saved files

In [ ]:
hist_LSTMDropout_train = pd.read_pickle("../final_data/hist_LSTMDropout_train")
hist_LSTMWeightDecay_train = pd.read_pickle("../final_data/hist_LSTMWeightDecay_train.pkl")
hist_LSTMHyperband_train = pd.read_pickle("../final_data/hist_LSTMHyperband_train.pkl")
hist_PeepholeDropout_train = pd.read_pickle("../final_data/hist_PeepholeDropout_model_train.pkl")
hist_PeepholeHyperband = pd.read_pickle("../final_data/hist_PeepholeHyperband_train.pkl")

In [ ]:
fig_simple_lstms, (ax1) = plt.subplots(figsize=(6, 5))
ax1.plot(hist_LSTMDropout_train['loss'], label='LSTMDropout_Loss', color='c')
ax1.plot(hist_LSTMDropout_train['val_loss'], label='LSTMDropout_Validation_Loss', color='blue')
ax1.plot(hist_LSTMWeightDecay_train['loss'][:250], label='LSTMWeightReg_Loss', color='mediumseagreen')
ax1.plot(hist_LSTMWeightDecay_train['val_loss'][:250], label='LSTMWeightReg_Validation_Loss', color='green')
ax1.plot(hist_PeepholeDropout_train['loss'], label='PeepholeDropout_Loss', color='indianred')
ax1.plot(hist_PeepholeDropout_train['val_loss'], label='PeepholeDropout_Validation_Loss', color='red')
ax1.set_ylim(0.0005, 0.004)
ax1.set(xlabel=' Epochs', ylabel='Score')
ax1.set_xlabel('Epochs', fontsize=15)
ax1.set_ylabel('Score', fontsize=15)
plt.yticks(fontsize=13)
plt.xticks(fontsize=13)
ax1.legend(loc='best')
ax1.legend(prop={'size': 12})
ax1.grid()

In [ ]:
fig_simple_lstms, ax = plt.subplots(figsize=(6, 5))
ax.plot(hist_LSTMHyperband_train['loss'], label='LSTMHyperband_Loss', color='c' )
ax.plot(hist_LSTMHyperband_train['val_loss'], label='LSTMHyperband_Validation_Loss', color='blue' )
ax.plot(hist_PeepholeHyperband['loss'], label='PeepholeHyperband_Loss', color='indianred' )
ax.plot(hist_PeepholeHyperband['val_loss'], label='PeepholeHyperband_Validation_Loss', color='red' )
ax.set_ylim(0.0, 0.004)
ax.set(xlabel=' Epochs', ylabel='Score')
ax.set_xlabel('Epochs', fontsize=15)
ax.set_ylabel('Score', fontsize=15)
plt.yticks(fontsize=13)
plt.xticks(fontsize=13)
ax.legend(loc='best')
ax.legend(prop={'size': 12})
ax.grid()

### Grafical Evaluation

In [ ]:
sns.set_style("white")

### If dropout lstm models was not run: Read saved files

In [ ]:
LSTMDropout_model_train = keras.models.load_model("../final_data/models/LSTMDropout_model_train")

LSTMDropout_model = keras.models.load_model("../final_data/models/LSTMDropout_model_test")

In [ ]:
y_pred_train = LSTMDropout_model_test.predict(all_train_x)
y_train_unsc = scaler_y.inverse_transform(all_train_y)
y_pred_train_unsc = scaler_y.inverse_transform(y_pred_train)
np.sqrt(mean_squared_error(y_train_unsc, y_pred_train_unsc))                       

In [ ]:
y_pred_valid = LSTMDropout_model_train.predict(valid_x)
y_valid_unsc = scaler_train_y.inverse_transform(valid_y)
y_pred_valid_unsc = scaler_train_y.inverse_transform(y_pred_valid)
np.sqrt(mean_squared_error(y_valid_unsc, y_pred_valid_unsc))                       

In [ ]:
y_pred_test = LSTMDropout_model_test.predict(test_x)
y_test_unsc = scaler_y.inverse_transform(test_y)
y_pred_test_unsc = scaler_y.inverse_transform(y_pred_test)
np.sqrt(mean_squared_error(y_test_unsc, y_pred_test_unsc))                       

### Scatter Plots

In [ ]:
fig, ax1 = plt.subplots(1, figsize=(5, 5),  sharex=True, sharey=True)
plt.xlim(550, 900)
plt.ylim(550, 900)

ax1.scatter( y_pred_train_unsc, y_train_unsc, c= 'dodgerblue',s=1)
ax1.plot([550,900],[550,900], c='black')
ax1.grid()
ax1.set_xlabel('Predicted Temperatures [°C] ', fontsize=15)
ax1.set_ylabel('Measured Temperature [°C]',fontsize=15)
ax1.set(yticks=[]) 
ax1.set(xticks=[]) 
ax1.xaxis.grid(True)
ax1.yaxis.grid(True)

In [ ]:
fig, ax1 = plt.subplots(1, figsize=(5, 5),  sharex=True, sharey=True)
plt.xlim(550, 900)
plt.ylim(550, 900)

ax1.scatter( y_pred_valid_unsc, y_valid_unsc, c= 'dodgerblue',s=1)
ax1.plot([550,900],[550,900], c='black')
ax1.grid()
ax1.set_xlabel('Predicted Temperatures [°C] ', fontsize=15)
ax1.set_ylabel('Measured Temperature [°C]',fontsize=15)
ax1.set(yticks=[]) 
ax1.set(xticks=[]) 
ax1.xaxis.grid(True)
ax1.yaxis.grid(True)

In [ ]:
fig, ax1 = plt.subplots(1,  figsize=(5, 5),  sharex=True, sharey=True)
plt.xlim(550, 900)
plt.ylim(550, 900)

ax1.scatter( y_pred_test_unsc, y_test_unsc,  c= 'dodgerblue',s=1)
ax1.plot([550,900],[550,900], c='black')
ax1.grid()
ax1.set_xlabel('Predicted Temperatures [°C]', fontsize=15)
ax1.set_ylabel('Measured Temperature [°C]',fontsize=15)
ax1.set(yticks=[]) 
ax1.set(xticks=[]) 
ax1.xaxis.grid(True)
ax1.yaxis.grid(True)

### looking at single sequences

In [ ]:
def lstm_format(data):
    length_cc4 = 15.42
    seq_len = 50
    features = []
    labels = []
    data_set = {}

    for _, group in data.groupby('seq_id'):
        for i in range(len(group)-(seq_len-1)):
            features.append(group[feature_keys].iloc[i:i+seq_len].to_numpy())   ##Achtung keys wieder ändern!
            labels.append(group[label_keys].iloc[i+(seq_len-1)].to_numpy())

    data_set['x'] = np.stack(features)
    data_set['y'] = np.stack(labels)
    return data_set

In [ ]:
test_seq = test_norm['seq_id'].drop_duplicates()

In [ ]:
x = {}
y = {}
for seq in test_seq:
    df = lstm_format(test_norm[test_norm['seq_id']==seq])
    x[seq]= df['x'].copy()
    y[seq] = df['y'].copy()

In [ ]:
prediction = {}
real_values = {}
rmse = {}
for seq in test_seq:
    prediction[seq] = scaler_y.inverse_transform(LSTMDropout_model.predict(x[seq]))
    real_values[seq] = scaler_y.inverse_transform(y[seq])
    rmse[seq] = np.sqrt(mean_squared_error( prediction[seq],  real_values[seq]))  

In [ ]:
error = []
for seq in test_seq:
    error.append((rmse[seq], seq))

In [ ]:
error

In [ ]:
sequences_name = '471949_str_1'

fig, (ax1) = plt.subplots(1, figsize=(6, 5),  sharex=True, sharey=True)
ax1.plot(real_values[sequences_name][:,:1], color='black', label='Measured Temperature')
ax1.plot(prediction[sequences_name][:,:1], color='blue', label='Predicted Temperature')
ax1.legend(loc='best')
ax1.grid()
ax1.set_xlabel('Minutes', fontsize=15)
ax1.set_ylabel('Temperature [°C]',fontsize=15)
ax1.set(yticks=[]) 
ax1.xaxis.grid(True)
ax1.yaxis.grid(True)
ax1.axhline(688, xmax=0.05, ls='--', color='red', label='Mean of Measured Temperature' )
ax1.axhline(688, xmin=0.93, ls='--', color='red', )
ax1.legend(prop={'size': 13})
print('RMSE:', rmse[sequences_name] )

In [ ]:
sequences_name = '471949_str_2'

fig, (ax1) = plt.subplots(1, figsize=(6, 5),  sharex=True, sharey=True)
ax1.plot(real_values[sequences_name][:,:1], color='black', label='Measured Temperature')
ax1.plot(prediction[sequences_name][:,:1], color='blue', label='Predicted Temperature')
ax1.legend(loc='best')
ax1.grid()
ax1.set_xlabel('Minutes', fontsize=15)
ax1.set_ylabel('Temperature [°C]',fontsize=15)
ax1.set(yticks=[]) 
ax1.xaxis.grid(True)
ax1.yaxis.grid(True)
ax1.axhline(680, ls='--', color='red', label='Mean of Measured Temperature' )
ax1.legend(prop={'size': 13})
print('RMSE:', rmse[sequences_name] )

In [ ]:
sequences_name = '473562_str_1'

fig, (ax1) = plt.subplots(1, figsize=(6, 5),  sharex=True, sharey=True)
ax1.plot(real_values[sequences_name][:,1:2], color='black', label='Measured Temperature')
ax1.plot(prediction[sequences_name][:,1:2], color='blue', label='Predicted Temperature')
ax1.legend(loc='best')
ax1.grid()
ax1.set_xlabel('Minutes', fontsize=15)
ax1.set_ylabel('Temperature [°C]',fontsize=15)
ax1.set(yticks=[]) 
ax1.xaxis.grid(True)
ax1.yaxis.grid(True)
ax1.axhline(719, xmax=0.05, ls='--', color='red', label='Mean of Measured Temperature' )
ax1.axhline(719, xmin=0.88, ls='--', color='red', )
ax1.legend(prop={'size': 13})


print('RMSE:', rmse[sequences_name] )

In [ ]:
sequences_name = '474185_str_1'

fig, (ax1) = plt.subplots(1, figsize=(6, 5),  sharex=True, sharey=True)
ax1.plot(real_values[sequences_name][:,:1], color='black', label='Measured Temperature')
ax1.plot(prediction[sequences_name][:,:1], color='blue', label='Predicted Temperature')
ax1.legend(loc='best')
ax1.grid()
ax1.set_xlabel('Minutes', fontsize=15)
ax1.set_ylabel('Temperature [°C]',fontsize=15)
ax1.set(yticks=[]) 
ax1.xaxis.grid(True)
ax1.yaxis.grid(True)
ax1.axhline(686, ls='--', color='red', label='Mean of Measured Temperature' )
ax1.legend(prop={'size': 13})

print('RMSE:', rmse[sequences_name] )

In [ ]:
sequences_name = '474261_str_1'

fig, (ax1) = plt.subplots(1, figsize=(6, 5),  sharex=True, sharey=True)
ax1.plot(real_values[sequences_name][:,1:2], color='black', label='Measured Temperature')
ax1.plot(prediction[sequences_name][:,1:2], color='blue', label='Predicted Temperature')
ax1.legend(loc='best')
ax1.grid()
ax1.set_xlabel('Minutes', fontsize=15)
ax1.set_ylabel('Temperature [°C]',fontsize=15)
ax1.set(yticks=[]) 
ax1.xaxis.grid(True)
ax1.yaxis.grid(True)
ax1.axhline(717, ls='--', color='red', label='Mean of Measured Temperature' )
ax1.legend(prop={'size': 13})

print('RMSE:', rmse[sequences_name] )

In [ ]:
sequences_name = '472885_str_2'

fig, (ax1) = plt.subplots(1, figsize=(6, 5),  sharex=True, sharey=True)
ax1.plot(real_values[sequences_name][:,:1], color='black', label='Measured Temperature')
ax1.plot(prediction[sequences_name][:,:1], color='blue', label='Predicted Temperature')
ax1.legend(loc='best')
ax1.grid()
ax1.set_xlabel('Minutes', fontsize=15)
ax1.set_ylabel('Temperature [°C]',fontsize=15)
ax1.set(yticks=[]) 
ax1.xaxis.grid(True)
ax1.yaxis.grid(True)
ax1.axhline(680, ls='--', color='red', label='Mean of Measured Temperature')
ax1.legend(prop={'size': 13})

print('RMSE:', rmse[sequences_name] )

In [ ]:
sequences_name = '472991_str_2'

fig, (ax1) = plt.subplots(1, figsize=(6, 5),  sharex=True, sharey=True)
ax1.plot(real_values[sequences_name][:,1:2], color='black', label='Measured Temperature')
ax1.plot(prediction[sequences_name][:,1:2], color='blue', label='Predicted Temperature')
ax1.set_xlabel('Minutes', fontsize=15)
ax1.set_ylabel('Temperature [°C]',fontsize=15)
ax1.set(yticks=[]) 
ax1.xaxis.grid(True)
ax1.yaxis.grid(True)
ax1.axhline(697, ls='--', color='red', label='Mean of Measured Temperature' )
ax1.legend(loc='best', prop={'size': 13})

print('RMSE:', rmse[sequences_name] )